In [1]:
from numcosmo_py import Ncm, Nc, GObject

import numpy as np
import math
import sys
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

from richness_mass_calib import create_richness_mass_calib

In [2]:
# #Settings
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [3]:
# Data
DC2_halos_m200c = fits.open(
    "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits"
)
dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos['redshift_true', 'mass_fof', 'm200c', 'richness']

In [4]:
rmdata = create_richness_mass_calib(dt_halos)
ascaso = Nc.ClusterMassAscaso()
lnrich_ext = Nc.ClusterMassLnrichExt()

mset = Ncm.MSet()
mset.set(lnrich_ext)

rmdata.m2lnL_val(mset)

13028468.257429378

In [5]:
dset = Ncm.Dataset.new()
dset.append_data(rmdata)

lh = Ncm.Likelihood.new(dset)
mset.param_set_all_ftype(Ncm.ParamType.FREE)
mset.prepare_fparam_map()


fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.run_restart(Ncm.FitRunMsgs.SIMPLE, 1.0e-3, 0.0, None, None)

#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
#  Minimum found with precision: |df|/f =  1.00000e-08 and |dx| =  1.00000e-05
#  Elapsed time: 00 days, 00:01:58.0247340
#  iteration            [005069]
#  function evaluations [005071]
#  gradient evaluations [000000]
#  degrees of freedom   [454460]
#  m2

In [6]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.60458890756452    [FREE]
#   -    muM1[01]:  0.872064918845062   [FREE]
#   -    muZ1[02]:  1.77402852218968    [FREE]
#   -    muM2[03]:  0.0417644607020131  [FREE]
#   -    muZ2[04]: -0.1407406182412     [FREE]
#   -    muMZ[05]:  0.142144935354152   [FREE]
#   -  sigma0[06]:  0.288050575207433   [FREE]
#   - sigmaM1[07]:  0.0166200968775334  [FREE]
#   - sigmaZ1[08]: -0.0169039689278136  [FREE]
#   - sigmaM2[09]:  0.0151639193313241  [FREE]
#   - sigmaZ2[10]:  0.00838580955437241 [FREE]
#   - sigmaMZ[11]:  0.0375484741041049  [FREE]
#   -     cut[12]:  20.9538542974676    [FRE

In [12]:
nwalkers = 2200

Ncm.func_eval_set_max_threads(4)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(4)
esmcmc.set_data_file("test_.fits")

esmcmc.start_run()
esmcmc.run(1000)  
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      4
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 2200.
#   Number of threads: 0004.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.6300157825435     [FREE]
#   -    muM1[01

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# NcmFitESMCMC: row 2197934 values:      119989.633873984       4.62418681125317      0.887659098981829       1.81459982887295     0.0446240055574888     -0.146687639820497      0.153294580393963      0.297460336906018       0.02505007477018    -0.0136193415085405      0.016783612455714    0.00833374380225495      0.038079426056421        5.4012659678002    -0.0148940659509639     -0.709710892036993
# NcmFitESMCMC: Catalog row 2197935: m2lnL =     120115.872390205, recalculated to     120115.872390205, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2197935 values:      120115.872390205        4.6091383947322      0.874746923296246       1.77414641512151     0.0422700774325891       -0.1397213657333      0.143086762703153      0.286478639187848     0.0134093634613078    -0.0284730298742206     0.0142369916570439     0.0113691949445212     0.0340948605526599       9.89592979479413      -2.49328405350641       -9.9705606686197
# NcmFitESMCMC: Catalog row 2197936: m2lnL =     119997.28

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



      4.61619324040294      0.880623162363606       1.78530339926582     0.0431449830607097     -0.140216287149784      0.144800228894657      0.291423654325768     0.0189393258625856    -0.0189725159431514      0.015515948306703    0.00999687008660197     0.0372368387647531       5.40931568447884    -0.0165102871188277     -0.732119629844561
# NcmFitESMCMC: Catalog row 2198066: m2lnL =     119985.765029323, recalculated to     119985.765029323, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2198066 values:      119985.765029323       4.62760629199802      0.890851036763538       1.79712176705635     0.0454579908888117     -0.138994082837658      0.150870484013753      0.290693304950184     0.0166506044208644    -0.0250418216040173     0.0149114912130632     0.0119497625345737     0.0353736344125049       5.45753659941575    -0.0329818297627084     -0.853088042445315
# NcmFitESMCMC: Catalog row 2198067: m2lnL =     119984.102301107, recalculated to     119984.102301107, diff = 0.00

...

      0.298184757201164        0.0248254070767    -0.0152692876432994     0.0165850024289005    0.00954357358282624     0.0370890290209127       5.37037040020365   -0.00229102538922895     -0.617130315961824
# NcmFitESMCMC: Catalog row 2199714: m2lnL =     119979.435027189, recalculated to     119979.435027189, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199714 values:      119979.435027189       4.63054064319817       0.89439088937755       1.79593927717851     0.0460847640408275     -0.136751289600822      0.149473852386727      0.290744398792097     0.0191017071097728    -0.0139088118073416     0.0157267669518353     0.0148839891863524     0.0398629703455457       5.41361996709637    -0.0131567523124423     -0.731805729013448
# NcmFitESMCMC: Catalog row 2199715: m2lnL =     119980.400450981, recalculated to     119980.400450981, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199715 values:      119980.400450981       4.62296312131503      0.889041686004279       1.80147

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



# NcmFitESMCMC: Catalog row 2199845: m2lnL =     119985.488621247, recalculated to     119985.488621247, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199845 values:      119985.488621247         4.626110368654      0.887325713190827       1.76688903798068     0.0443339640600727     -0.133052483842373      0.141119850766769      0.296590120439817      0.022200718783521     -0.022638023783742     0.0161391431827058     0.0105221484893039     0.0366729065936905       5.38573817480855   -0.00423048996993723     -0.660283057726663
# NcmFitESMCMC: Catalog row 2199846: m2lnL =     119979.122225629, recalculated to     119979.122225629, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199846 values:      119979.122225629       4.62089322912975        0.8884342788328       1.80904620869245     0.0452988394618551     -0.137028491179942      0.154631868839382      0.288338196752989     0.0180001634412727    -0.0148089376418587     0.0155783968993218    0.00759692761752149     0.03814126

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  -0.00751558092644217     -0.647442816638834
# NcmFitESMCMC: Catalog row 2199980: m2lnL =     119986.396501751, recalculated to     119986.396501751, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199980 values:      119986.396501751       4.63460431468263      0.897798555026077       1.78628591008074     0.0468286184711978     -0.140710646900653      0.146479569512375      0.299793069532455     0.0253695485395963     -0.015867197607657     0.0167511712339971     0.0109356241823686     0.0383913055216832       5.40449985592437    -0.0113424987663075     -0.704562185875305
# NcmFitESMCMC: Catalog row 2199981: m2lnL =     119987.463256932, recalculated to     119987.463256932, diff = 0.00000e+00, SUCCEEDED!
# NcmFitESMCMC: row 2199981 values:      119987.463256932       4.62487016255182      0.889554425176702       1.80357474353334     0.0453388575106318     -0.132726700412195      0.153282317032608      0.294544121363657     0.0205938077692898    -0.0234933321748253      0.0158452

In [13]:
esmcmc.mean_covar()
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#      mu[07000:00] =  4.622       +/-  0.01269     |  1           |  0.8749      |  0.2358      |  0.7528      |  0.07272     |  0.2655      |  0.2717      |  0.2987      |  0.1419      |  0.2363      | -0.07977     |  0.0782      | -0.4211      | -0.06457     |  0.1895      |
#    muM1[07000:01] =  0.8871      +/-  0.00928     |  0.8749      |  1           |  0.4433      |  0.9474      |  0.1399      |  0.4472      |  0.3244      |  0.3681      |  0.121       |  0.3189      | -0.1201      |  0.04084     | -0.5203      | -0.09173     |  0.2191      |
#    muZ1[07000:02] =  1.79        +/-  

In [14]:
fit.log_covar()

#----------------------------------------------------------------------------------
# NcmMSet parameters covariance matrix
#                                                   ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#      mu[07000:00] =  4.622       +/-  0.01269     |  1           |  0.8749      |  0.2358      |  0.7528      |  0.07272     |  0.2655      |  0.2717      |  0.2987      |  0.1419      |  0.2363      | -0.07977     |  0.0782      | -0.4211      | -0.06457     |  0.1895      |
#    muM1[07000:01] =  0.8871      +/-  0.00928     |  0.8749      |  1           |  0.4433      |  0.9474      |  0.1399      |  0.4472      |  0.3244      |  0.3681      |  0.121       |  0.3189      | -0.1201      |  0.04084     | -0.5203      | -0.09173     |  0.2191      |
#    muZ1[07000:02] =  1.79        +/-  

In [15]:
fit.log_info()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[07000]:
#   - NcClusterMass : LnrichExt Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -      mu[00]:  4.62235290158276    [FREE]
#   -    muM1[01]:  0.88711297078257    [FREE]
#   -    muZ1[02]:  1.78987317526121    [FREE]
#   -    muM2[03]:  0.0446680706658167  [FREE]
#   -    muZ2[04]: -0.139975395256496   [FREE]
#   -    muMZ[05]:  0.147095254903743   [FREE]
#   -  sigma0[06]:  0.292163459008121   [FREE]
#   - sigmaM1[07]:  0.0196627036414719  [FREE]
#   - sigmaZ1[08]: -0.0149205695211413  [FREE]
#   - sigmaM2[09]:  0.0157075140310971  [FREE]
#   - sigmaZ2[10]:  0.00945577317556722 [FREE]
#   - sigmaMZ[11]:  0.0383559981783699  [FREE]
#   -     cut[12]:  8.48270947638978    [FRE